# Lab 3.2 - Agentic AI with CrewAI + watsonx.ai

This notebook demonstrates how to build a multi-agent system using CrewAI with watsonx.ai as the LLM backend.

## What You'll Learn

- How to integrate watsonx.ai models with CrewAI
- Creating custom tools for agents (RAG service and calculator)
- Building a collaborative agent system
- Using agents to solve complex tasks

## Architecture

1. **watsonx.ai** - IBM's foundation model platform (Granite models)
2. **CrewAI** - Multi-agent orchestration framework
3. **Custom Tools** - RAG service and calculator tools
4. **Accelerator RAG API** - Backend RAG service for knowledge retrieval

---

## 1. Setup and Installation

### Google Colab Compatibility

This notebook is designed to work both locally and in Google Colab.

In [ ]:
# Check if running in Google Colab
try:
    import google.colab
    IN_COLAB = True
    print("✓ Running in Google Colab")
except ImportError:
    IN_COLAB = False
    print("✓ Running in local environment")

In [ ]:
# Install required packages
!pip install -q crewai crewai-tools requests "ibm-watsonx-ai>=1.1.22" langchain-ibm

## 2. Configure watsonx.ai Credentials

To use watsonx.ai, you need:

1. **API Key** - Get it from [IBM Cloud](https://cloud.ibm.com/iam/apikeys)
2. **Project ID** - From your watsonx.ai project
3. **URL** - Regional endpoint (default: us-south)

### How to Get Your Credentials

1. Go to [IBM Cloud](https://cloud.ibm.com)
2. Navigate to watsonx.ai
3. Create or open a project
4. Copy your Project ID from the project settings
5. Create an API key from IBM Cloud IAM

In [ ]:
import os
from getpass import getpass

# Configuration for watsonx.ai
WATSONX_URL = os.getenv("WATSONX_URL", "https://us-south.ml.cloud.ibm.com")

if not os.getenv("WATSONX_APIKEY"):
    WATSONX_APIKEY = getpass("Enter your watsonx.ai API Key: ")
else:
    WATSONX_APIKEY = os.getenv("WATSONX_APIKEY")

if not os.getenv("WATSONX_PROJECT_ID"):
    WATSONX_PROJECT_ID = getpass("Enter your watsonx.ai Project ID: ")
else:
    WATSONX_PROJECT_ID = os.getenv("WATSONX_PROJECT_ID")

# Model configuration
LLM_MODEL_ID = os.getenv("LLM_MODEL_ID", "ibm/granite-3-8b-instruct")

# Accelerator API URL (set this to your RAG service endpoint)
ACCELERATOR_API_URL = os.getenv("ACCELERATOR_API_URL", "http://localhost:8000/ask")

print("✓ Configuration loaded")
print(f"  Model: {LLM_MODEL_ID}")
print(f"  URL: {WATSONX_URL}")
print(f"  RAG API: {ACCELERATOR_API_URL}")

## 3. Initialize watsonx.ai LLM for CrewAI

CrewAI can work with LangChain-compatible LLMs. We'll use the IBM watsonx.ai integration through LangChain.

In [ ]:
from langchain_ibm import WatsonxLLM
from ibm_watsonx_ai.foundation_models.utils.enums import DecodingMethods

# Configure model parameters
model_params = {
    "decoding_method": "greedy",
    "max_new_tokens": 1000,
    "min_new_tokens": 1,
    "temperature": 0.5,
    "top_k": 50,
    "top_p": 1
}

# Initialize watsonx.ai LLM
watsonx_llm = WatsonxLLM(
    model_id=LLM_MODEL_ID,
    url=WATSONX_URL,
    apikey=WATSONX_APIKEY,
    project_id=WATSONX_PROJECT_ID,
    params=model_params
)

print("✓ watsonx.ai LLM initialized successfully")

## 4. Define Custom Tools

We'll create two tools for our agents:

1. **RAG Service Tool** - Queries the accelerator RAG API for knowledge retrieval
2. **Calculator Tool** - Performs safe arithmetic calculations

These tools give our agents the ability to access external knowledge and perform computations.

In [ ]:
import json
import ast
import operator as op
from typing import Any
import requests
from crewai_tools import tool

@tool("rag_service_tool")
def rag_service_tool(question: str) -> str:
    """
    Query the RAG (Retrieval-Augmented Generation) service to answer questions
    based on enterprise knowledge base.
    
    Use this tool when you need to:
    - Answer questions about specific documents or knowledge
    - Retrieve factual information from the knowledge base
    - Get context-aware responses based on enterprise data
    
    Args:
        question: The question to ask the RAG service
    
    Returns:
        Formatted string containing the answer and citations
    """
    try:
        payload = {"question": question}
        resp = requests.post(ACCELERATOR_API_URL, json=payload, timeout=60)
        resp.raise_for_status()
        data = resp.json()
        
        answer = data.get("answer") or data.get("result") or "No answer available"
        citations = data.get("citations") or data.get("chunks") or []
        
        result = f"ANSWER: {answer}\n"
        if citations:
            result += f"\nCITATIONS: {json.dumps(citations, indent=2)}"
        
        return result
    except Exception as e:
        return f"Error calling RAG service: {str(e)}"


# Safe calculator implementation using AST
_allowed_operators = {
    ast.Add: op.add,
    ast.Sub: op.sub,
    ast.Mult: op.mul,
    ast.Div: op.truediv,
    ast.Pow: op.pow,
}


def _eval_ast(node):
    """Safely evaluate an AST node."""
    if isinstance(node, ast.Num):  # Python 3.7 compatibility
        return node.n
    if isinstance(node, ast.Constant):  # Python 3.8+
        return node.value
    if isinstance(node, ast.BinOp) and type(node.op) in _allowed_operators:
        return _allowed_operators[type(node.op)](_eval_ast(node.left), _eval_ast(node.right))
    if isinstance(node, ast.UnaryOp) and isinstance(node.op, (ast.UAdd, ast.USub)):
        value = _eval_ast(node.operand)
        return +value if isinstance(node.op, ast.UAdd) else -value
    raise ValueError("Unsupported expression")


@tool("calculator_tool")
def calculator_tool(expression: str) -> str:
    """
    Safely evaluate arithmetic expressions.
    
    Use this tool when you need to:
    - Perform mathematical calculations
    - Evaluate arithmetic expressions
    - Do numerical computations
    
    Supports: +, -, *, /, ** (power)
    Examples: '2 + 2', '10 * (5 + 3)', '2 ** 8'
    
    Args:
        expression: Mathematical expression to evaluate
    
    Returns:
        Result of the calculation or error message
    """
    try:
        parsed = ast.parse(expression, mode="eval")
        result = _eval_ast(parsed.body)
        return f"Result: {result}"
    except Exception as e:
        return f"Error evaluating expression: {str(e)}"


print("✓ Custom tools defined successfully")

## 5. Create CrewAI Agents

Now we'll create specialized agents:

1. **Research Agent** - Uses RAG service to find information
2. **Calculator Agent** - Handles mathematical computations

Each agent has a specific role, goal, and backstory that guides its behavior.

In [ ]:
from crewai import Agent, Task, Crew, Process

# Research Agent - specializes in information retrieval
research_agent = Agent(
    role="Research Specialist",
    goal="Find accurate and relevant information from the knowledge base to answer user questions",
    backstory="""
        You are an expert research specialist with deep knowledge of watsonx.ai and 
        enterprise AI systems. You excel at finding relevant information and providing 
        well-cited, accurate answers. You always use the RAG service to ground your 
        responses in factual data.
    """,
    tools=[rag_service_tool],
    llm=watsonx_llm,
    verbose=True,
    allow_delegation=False
)

# Calculator Agent - specializes in mathematical operations
calculator_agent = Agent(
    role="Mathematics Specialist",
    goal="Perform accurate mathematical calculations and explain the results",
    backstory="""
        You are a mathematics expert who excels at performing calculations and 
        explaining mathematical concepts. You use the calculator tool for all 
        numerical computations to ensure accuracy.
    """,
    tools=[calculator_tool],
    llm=watsonx_llm,
    verbose=True,
    allow_delegation=False
)

# Support Agent - orchestrates and delegates to specialists
support_agent = Agent(
    role="Workshop Support Coordinator",
    goal="Help users by delegating questions to the appropriate specialist agent",
    backstory="""
        You are a helpful coordinator who understands when to use the RAG service 
        for knowledge questions and when to use the calculator for math problems. 
        You have access to both tools and can decide which is most appropriate.
    """,
    tools=[rag_service_tool, calculator_tool],
    llm=watsonx_llm,
    verbose=True,
    allow_delegation=True
)

print("✓ Agents created successfully")

## 6. Define Tasks and Create Crew

We'll create a task that can be handled by our support agent, which will use the appropriate tool or delegate to specialist agents as needed.

In [ ]:
def create_support_task(question: str) -> Task:
    """
    Create a task for the support agent to handle a user question.
    
    Args:
        question: The user's question
    
    Returns:
        Task object configured for the support agent
    """
    return Task(
        description=f"""
            Answer the following user question: {question}
            
            Guidelines:
            - For factual or knowledge-based questions, use the RAG service tool
            - For mathematical calculations, use the calculator tool
            - Provide clear, concise answers with proper citations when applicable
            - If you use a tool, explain what tool you used and why
        """,
        expected_output="A clear, helpful answer with explanation of tools used",
        agent=support_agent,
    )


# Create the crew
crew = Crew(
    agents=[support_agent, research_agent, calculator_agent],
    tasks=[],  # Tasks will be added dynamically
    process=Process.sequential,
    verbose=True
)

print("✓ Crew initialized")

## 7. Test the Multi-Agent System

Let's test our crew with different types of questions to see how it handles them.

### Test 1: Knowledge Question (Should use RAG service)

In [ ]:
question_1 = "What is Retrieval-Augmented Generation (RAG) and why is it important?"

print("=" * 80)
print(f"USER QUESTION: {question_1}")
print("=" * 80)

task_1 = create_support_task(question_1)
crew.tasks = [task_1]
result_1 = crew.kickoff()

print("\n" + "=" * 80)
print("FINAL ANSWER:")
print("=" * 80)
print(result_1)

### Test 2: Mathematical Question (Should use calculator)

In [ ]:
question_2 = "Calculate: (15 + 25) * 3 - 10"

print("=" * 80)
print(f"USER QUESTION: {question_2}")
print("=" * 80)

task_2 = create_support_task(question_2)
crew.tasks = [task_2]
result_2 = crew.kickoff()

print("\n" + "=" * 80)
print("FINAL ANSWER:")
print("=" * 80)
print(result_2)

### Test 3: Mixed Question

In [ ]:
question_3 = "What are the benefits of using watsonx.ai for enterprise AI?"

print("=" * 80)
print(f"USER QUESTION: {question_3}")
print("=" * 80)

task_3 = create_support_task(question_3)
crew.tasks = [task_3]
result_3 = crew.kickoff()

print("\n" + "=" * 80)
print("FINAL ANSWER:")
print("=" * 80)
print(result_3)

## 8. Interactive Question Answering

Now you can ask your own questions!

In [ ]:
def ask_crew(question: str):
    """
    Ask a question to the crew and get an answer.
    
    Args:
        question: Your question
    
    Returns:
        The crew's answer
    """
    print("=" * 80)
    print(f"QUESTION: {question}")
    print("=" * 80)
    
    task = create_support_task(question)
    crew.tasks = [task]
    result = crew.kickoff()
    
    print("\n" + "=" * 80)
    print("ANSWER:")
    print("=" * 80)
    print(result)
    
    return result


# Try it out!
# ask_crew("Your question here")

## 9. Summary and Key Takeaways

### What We Learned

1. **watsonx.ai Integration**: We successfully integrated IBM's Granite models with CrewAI using LangChain
2. **Custom Tools**: We created custom tools for RAG service and calculator functionality
3. **Multi-Agent Architecture**: We built specialized agents with distinct roles and capabilities
4. **Agent Collaboration**: Agents can work together and delegate tasks based on their expertise

### Key Components

- **watsonx.ai**: Foundation model platform providing Granite LLMs
- **CrewAI**: Multi-agent orchestration framework
- **Custom Tools**: Extensions that give agents special capabilities
- **RAG Service**: External knowledge retrieval system

### Best Practices

1. **Clear Agent Roles**: Define specific roles and responsibilities for each agent
2. **Tool Selection**: Give agents only the tools they need for their role
3. **Delegation**: Use delegation strategically to leverage specialist agents
4. **Error Handling**: Always include proper error handling in custom tools
5. **Documentation**: Provide clear docstrings for tools so agents understand when to use them

### Next Steps

- Try the LangGraph notebook for a different agent architecture
- Experiment with the Langflow visual builder
- Create your own custom tools and agents
- Build more complex multi-agent workflows

---

**Course**: Multi-Agent Systems with watsonx.ai  
**Lab**: 3.2 - CrewAI Integration  
**Platform**: Compatible with Google Colab and local environments